Extracts Data from the Precriber Summary Information and Detailed Data of the Medicare Part D dataset.

In [1]:
import pandas as pd
import requests
import numpy as np

In [2]:
pd.set_option("display.max_columns", 150) #increase column options

In [3]:
md_base_url = "https://data.cms.gov/resource/" #Base URL to access Databases

Manipulation of Detailed Data (Database 1) 
Years: 2017

In [4]:
url_ID = "77gb-8z53"
dd_url = md_base_url + url_ID
print(dd_url)

https://data.cms.gov/resource/77gb-8z53


In [5]:
dd_provider = requests.get(dd_url, params={"$select":"nppes_provider_state,count(nppes_provider_state)", "$group": "nppes_provider_state"})
dd_provider_df = pd.read_json(dd_provider.content)
dd_provider_df[dd_provider_df.count_nppes_provider_state < 50000]
#retrieves the data based on providers per region

,count_nppes_provider_state,nppes_provider_state
0,6,AS
1,40,AA
2,222,XX
3,565,AE
4,1417,ZZ
5,319,AP
6,278,MP
7,1762,GU
8,37925,WY
9,3373,VI


In [6]:
dd_requests = requests.get(dd_url, params={"$where": "nppes_provider_state='DC'", "$limit": 50000})
#calls data of a specific state 

In [7]:
dd_requests.headers

{'Server': 'nginx', 'Date': 'Tue, 09 Jul 2019 18:34:19 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Socrata-RequestId': '4kci295q00yixpyrjh7roi3hf', 'Access-Control-Allow-Origin': '*', 'ETag': '"a59894aa67eab310d94159a04b9b406a--gzip"', 'Last-Modified': 'Wed, 01 May 2019 16:24:35 UTC', 'X-SODA2-Warning': 'X-SODA2-Fields, X-SODA2-Types, and X-SODA2-Legacy-Types are deprecated', 'X-SODA2-Fields': '["drug_name","total_day_supply","ge65_suppress_flag","npi","nppes_provider_first_name","total_claim_count_ge65","generic_name","total_30_day_fill_count_ge65","specialty_desc","total_day_supply_ge65","bene_count_ge65_suppress_flag","total_claim_count","nppes_provider_city","total_30_day_fill_count","bene_count_ge65","nppes_provider_state","nppes_provider_last_org_name","total_drug_cost","description_flag","total_drug_cost_ge65","bene_count"]', 'X-SODA2-Types': '["text","number","text","text","text","number","text","number

In [8]:
dd_df = pd.read_json(dd_requests.content)
dd_df = dd_df.sort_values(by=["npi","specialty_desc","drug_name"]) #read json & sort
dd_df.head(10)


,bene_count,bene_count_ge65,bene_count_ge65_suppress_flag,description_flag,drug_name,ge65_suppress_flag,generic_name,npi,nppes_provider_city,nppes_provider_first_name,nppes_provider_last_org_name,nppes_provider_state,specialty_desc,total_30_day_fill_count,total_30_day_fill_count_ge65,total_claim_count,total_claim_count_ge65,total_day_supply,total_day_supply_ge65,total_drug_cost,total_drug_cost_ge65
42422,NaN,NaN,#,S,ADVAIR DISKUS,#,FLUTICASONE/SALMETEROL,1003005661,WASHINGTON,ADEFOLAJU,OKETOKUN,DC,General Practice,15.0,NaN,15,NaN,450,NaN,4692.26,NaN
18892,NaN,NaN,*,S,ALENDRONATE SODIUM,*,ALENDRONATE SODIUM,1003005661,WASHINGTON,ADEFOLAJU,OKETOKUN,DC,General Practice,17.1,NaN,17,NaN,502,NaN,203.92,NaN
8223,NaN,NaN,*,S,ALLOPURINOL,NaN,ALLOPURINOL,1003005661,WASHINGTON,ADEFOLAJU,OKETOKUN,DC,General Practice,22.0,22.0,22,22.0,660,660.0,158.63,158.63
25251,24.0,NaN,#,S,AMLODIPINE BESYLATE,NaN,AMLODIPINE BESYLATE,1003005661,WASHINGTON,ADEFOLAJU,OKETOKUN,DC,General Practice,142.5,82.0,138,78.0,4275,2460.0,617.11,351.15
36992,NaN,NaN,*,S,AMMONIUM LACTATE,*,AMMONIUM LACTATE,1003005661,WASHINGTON,ADEFOLAJU,OKETOKUN,DC,General Practice,14.0,NaN,14,NaN,384,NaN,419.47,NaN
20072,11.0,NaN,*,S,AMOXICILLIN-CLAVULANATE POTASS,*,AMOXICILLIN/POTASSIUM CLAV,1003005661,WASHINGTON,ADEFOLAJU,OKETOKUN,DC,General Practice,12.0,NaN,12,NaN,111,NaN,144.49,NaN
2264,NaN,NaN,#,S,ATENOLOL,#,ATENOLOL,1003005661,WASHINGTON,ADEFOLAJU,OKETOKUN,DC,General Practice,32.0,NaN,32,NaN,960,NaN,233.06,NaN
31241,NaN,NaN,*,S,ATORVASTATIN CALCIUM,NaN,ATORVASTATIN CALCIUM,1003005661,WASHINGTON,ADEFOLAJU,OKETOKUN,DC,General Practice,62.3,22.0,60,20.0,1868,660.0,573.17,150.57
31435,11.0,NaN,*,S,AZITHROMYCIN,*,AZITHROMYCIN,1003005661,WASHINGTON,ADEFOLAJU,OKETOKUN,DC,General Practice,12.0,NaN,12,NaN,57,NaN,70.76,NaN
18940,NaN,NaN,#,S,CARVEDILOL,#,CARVEDILOL,1003005661,WASHINGTON,ADEFOLAJU,OKETOKUN,DC,General Practice,21.2,NaN,20,NaN,620,NaN,111.12,NaN


In [9]:
npi_drug_cross_df = pd.crosstab(dd_df["npi"], dd_df["drug_name"] , values=dd_df["total_claim_count"], 
                                  aggfunc=np.sum) #uses drug_name which is the brand names
npi_drug_cross_df = npi_drug_cross_df.fillna(0)
npi_drug_cross_df.head(10)

drug_name,ABACAVIR,ABACAVIR-LAMIVUDINE,ABACAVIR-LAMIVUDINE-ZIDOVUDINE,ABILIFY MAINTENA,ACAMPROSATE CALCIUM,ACARBOSE,ACCUPRIL,ACEBUTOLOL HCL,ACETAMINOPHEN-CODEINE,ACETAZOLAMIDE,ACTEMRA,ACYCLOVIR,ADACEL TDAP,ADAPALENE,ADCIRCA,ADDERALL,ADDERALL XR,ADEFOVIR DIPIVOXIL,ADEMPAS,ADVAIR DISKUS,ADVAIR HFA,AGGRENOX,ALBUTEROL SULFATE,ALCLOMETASONE DIPROPIONATE,ALCOHOL PADS,ALCOHOL PREP PADS,ALENDRONATE SODIUM,ALFUZOSIN HCL ER,ALLOPURINOL,ALOSETRON HCL,ALPHAGAN P,ALPRAZOLAM,ALPRAZOLAM ER,ALTAVERA,AMANTADINE,AMBIEN CR,AMIKACIN SULFATE,AMILORIDE HCL,AMIODARONE HCL,AMITIZA,AMITRIPTYLINE HCL,AMLODIPINE BESYLATE,AMLODIPINE BESYLATE-BENAZEPRIL,AMLODIPINE-ATORVASTATIN,AMLODIPINE-OLMESARTAN,AMLODIPINE-VALSARTAN,AMLODIPINE-VALSARTAN-HCTZ,AMMONIUM LACTATE,AMOXAPINE,AMOXICILLIN,AMOXICILLIN-CLAVULANATE POTASS,AMPICILLIN TRIHYDRATE,AMPYRA,ANAGRELIDE HCL,ANASTROZOLE,ANDROGEL,ANORO ELLIPTA,APRISO,APTIOM,ARANESP,ARIPIPRAZOLE,ARISTADA,ARMODAFINIL,ARMOUR THYROID,ARNUITY ELLIPTA,ASCOMP WITH CODEINE,ASMANEX,ASPIRIN-DIPYRIDAMOLE ER,ASTAGRAF XL,ATENOLOL,ATENOLOL-CHLORTHALIDONE,ATOMOXETINE HCL,ATORVASTATIN CALCIUM,ATOVAQUONE,ATOVAQUONE-PROGUANIL HCL,...,UNITHROID,UPTRAVI,URSODIOL,VAGIFEM,VALACYCLOVIR,VALGANCICLOVIR HCL,VALIUM,VALPROIC ACID,VALSARTAN,VALSARTAN-HYDROCHLOROTHIAZIDE,VANCOMYCIN,VANCOMYCIN HCL,VASCEPA,VELPHORO,VEMLIDY,VENCLEXTA,VENLAFAXINE HCL,VENLAFAXINE HCL ER,VENTOLIN HFA,VERAPAMIL ER,VERAPAMIL ER PM,VERAPAMIL HCL,VERAPAMIL SR,VESICARE,VIBERZI,VICTOZA 2-PAK,VICTOZA 3-PAK,VIGAMOX,VIIBRYD,VIMPAT,VIRACEPT,VIREAD,VIVELLE-DOT,VIVITROL,VOLTAREN,VOTRIENT,VRAYLAR,VYVANSE,WARFARIN SODIUM,WATER,WELCHOL,WELLBUTRIN XL,XALATAN,XALKORI,XANAX,XARELTO,XELJANZ,XELJANZ XR,XENAZINE,XIFAXAN,XIGDUO XR,XOLAIR,XOPENEX HFA,XTANDI,YUVAFEM,ZALEPLON,ZARXIO,ZENPEP,ZEPATIER,ZERBAXA,ZETIA,ZIDOVUDINE,ZINBRYTA,ZIOPTAN,ZIPRASIDONE HCL,ZOLOFT,ZOLPIDEM TARTRATE,ZOLPIDEM TARTRATE ER,ZONISAMIDE,ZOSTAVAX,ZUBSOLV,ZYDELIG,ZYPREXA,ZYPREXA RELPREVV,ZYTIGA
npi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1003005661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,138.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,60.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003012063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,176.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,101.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003042474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003059437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [33]:
npi_drug_cross_df.shape

(2984, 1109)

In [10]:
npi_drugcost_cross_df = pd.crosstab(dd_df["npi"], dd_df["drug_name"] , values=dd_df["total_drug_cost"], 
                                  aggfunc=np.sum)
npi_drugcost_cross_df = npi_drugcost_cross_df.fillna(0)
npi_drugcost_cross_df.head(10)

drug_name,ABACAVIR,ABACAVIR-LAMIVUDINE,ABACAVIR-LAMIVUDINE-ZIDOVUDINE,ABILIFY MAINTENA,ACAMPROSATE CALCIUM,ACARBOSE,ACCUPRIL,ACEBUTOLOL HCL,ACETAMINOPHEN-CODEINE,ACETAZOLAMIDE,ACTEMRA,ACYCLOVIR,ADACEL TDAP,ADAPALENE,ADCIRCA,ADDERALL,ADDERALL XR,ADEFOVIR DIPIVOXIL,ADEMPAS,ADVAIR DISKUS,ADVAIR HFA,AGGRENOX,ALBUTEROL SULFATE,ALCLOMETASONE DIPROPIONATE,ALCOHOL PADS,ALCOHOL PREP PADS,ALENDRONATE SODIUM,ALFUZOSIN HCL ER,ALLOPURINOL,ALOSETRON HCL,ALPHAGAN P,ALPRAZOLAM,ALPRAZOLAM ER,ALTAVERA,AMANTADINE,AMBIEN CR,AMIKACIN SULFATE,AMILORIDE HCL,AMIODARONE HCL,AMITIZA,AMITRIPTYLINE HCL,AMLODIPINE BESYLATE,AMLODIPINE BESYLATE-BENAZEPRIL,AMLODIPINE-ATORVASTATIN,AMLODIPINE-OLMESARTAN,AMLODIPINE-VALSARTAN,AMLODIPINE-VALSARTAN-HCTZ,AMMONIUM LACTATE,AMOXAPINE,AMOXICILLIN,AMOXICILLIN-CLAVULANATE POTASS,AMPICILLIN TRIHYDRATE,AMPYRA,ANAGRELIDE HCL,ANASTROZOLE,ANDROGEL,ANORO ELLIPTA,APRISO,APTIOM,ARANESP,ARIPIPRAZOLE,ARISTADA,ARMODAFINIL,ARMOUR THYROID,ARNUITY ELLIPTA,ASCOMP WITH CODEINE,ASMANEX,ASPIRIN-DIPYRIDAMOLE ER,ASTAGRAF XL,ATENOLOL,ATENOLOL-CHLORTHALIDONE,ATOMOXETINE HCL,ATORVASTATIN CALCIUM,ATOVAQUONE,ATOVAQUONE-PROGUANIL HCL,...,UNITHROID,UPTRAVI,URSODIOL,VAGIFEM,VALACYCLOVIR,VALGANCICLOVIR HCL,VALIUM,VALPROIC ACID,VALSARTAN,VALSARTAN-HYDROCHLOROTHIAZIDE,VANCOMYCIN,VANCOMYCIN HCL,VASCEPA,VELPHORO,VEMLIDY,VENCLEXTA,VENLAFAXINE HCL,VENLAFAXINE HCL ER,VENTOLIN HFA,VERAPAMIL ER,VERAPAMIL ER PM,VERAPAMIL HCL,VERAPAMIL SR,VESICARE,VIBERZI,VICTOZA 2-PAK,VICTOZA 3-PAK,VIGAMOX,VIIBRYD,VIMPAT,VIRACEPT,VIREAD,VIVELLE-DOT,VIVITROL,VOLTAREN,VOTRIENT,VRAYLAR,VYVANSE,WARFARIN SODIUM,WATER,WELCHOL,WELLBUTRIN XL,XALATAN,XALKORI,XANAX,XARELTO,XELJANZ,XELJANZ XR,XENAZINE,XIFAXAN,XIGDUO XR,XOLAIR,XOPENEX HFA,XTANDI,YUVAFEM,ZALEPLON,ZARXIO,ZENPEP,ZEPATIER,ZERBAXA,ZETIA,ZIDOVUDINE,ZINBRYTA,ZIOPTAN,ZIPRASIDONE HCL,ZOLOFT,ZOLPIDEM TARTRATE,ZOLPIDEM TARTRATE ER,ZONISAMIDE,ZOSTAVAX,ZUBSOLV,ZYDELIG,ZYPREXA,ZYPREXA RELPREVV,ZYTIGA
npi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1003005661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4692.26,0.0,0.0,0.0,0.0,0.0,0.0,203.92,0.0,158.63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,617.11,0.0,0.0,0.0,0.0,0.0,419.47,0.0,0.00,144.49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,233.06,0.0,0.0,573.17,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,542.00,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2665.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9002.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003012063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,252.21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1100.05,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,1150.06,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,568.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003042474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,807.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

Manipulation of Prescriber Summary Data (Database 2)
Year: 2017


In [11]:
url_ID_2 = "psut-35i4"
ps_url = md_base_url + url_ID_2
print(ps_url)

https://data.cms.gov/resource/psut-35i4


In [12]:
ps_provider = requests.get(ps_url, params={"$select":"nppes_provider_state,count(nppes_provider_state)", "$group": "nppes_provider_state"})
ps_provider_df = pd.read_json(ps_provider.content)
ps_provider_df[ps_provider_df.count_nppes_provider_state < 50000]
#ps_provider_df.head(10)
#retrieves the data based on providers per region

,count_nppes_provider_state,nppes_provider_state
0,4,AS
1,11,XX
2,88,AE
3,27,MP
4,176,VI
5,57,AP
6,166,ZZ
7,180,GU
8,1816,WY
9,23,AA


In [13]:
ps_requests = requests.get(ps_url, params={"$where": "nppes_provider_state='DC'", "$limit": 50000})
#calls data of a specific state 

In [14]:
ps_requests.headers

{'Server': 'nginx', 'Date': 'Tue, 09 Jul 2019 18:36:11 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Socrata-RequestId': '385unz0r0zxltffhwgk22t410', 'Access-Control-Allow-Origin': '*', 'ETag': '"d25d1bced793903cd2254b3e95e654b--gzip"', 'Last-Modified': 'Wed, 01 May 2019 16:41:34 UTC', 'X-SODA2-Warning': 'X-SODA2-Fields, X-SODA2-Types, and X-SODA2-Legacy-Types are deprecated', 'X-SODA2-Fields': '["antipsych_claim_count_ge65","specialty_description","la_opioid_claim_count","beneficiary_average_risk_score","pdp_suppress_flag","beneficiary_age_75_84_count","generic_claim_count","antipsych_ge65_suppress_flag","nppes_provider_gender","nppes_provider_city","brand_claim_count","nppes_credentials","nppes_provider_street1","la_opioid_day_supply","nppes_provider_street2","opioid_bene_count","brand_suppress_flag","nppes_provider_mi","generic_suppress_flag","lis_drug_cost","beneficiary_age_greater_84_count","total_claim_coun

In [20]:
ps_df = pd.read_json(ps_requests.content)
ps_df = ps_df.sort_values(by=["npi","specialty_description"]) #read json & sort
ps_df.head(10)


,anti_psych_bene_count_ge65,antibiotic_bene_count,antibiotic_claim_count,antibiotic_drug_cost,antipsych_bene_ge65_suppress_flg,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,antipsych_ge65_suppress_flag,average_age_of_beneficiaries,bene_count,bene_count_ge65,bene_count_ge65_suppress_flag,beneficiary_age_65_74_count,beneficiary_age_75_84_count,beneficiary_age_greater_84_count,beneficiary_age_less_65_count,beneficiary_average_risk_score,beneficiary_dual_count,beneficiary_female_count,beneficiary_male_count,beneficiary_nondual_count,beneficiary_race_asian_pi_count,beneficiary_race_black_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_race_white_count,brand_claim_count,brand_drug_cost,brand_suppress_flag,description_flag,ge65_suppress_flag,generic_claim_count,generic_drug_cost,generic_suppress_flag,la_opioid_bene_count,la_opioid_claim_count,la_opioid_day_supply,la_opioid_drug_cost,la_opioid_prescriber_rate,lis_claim_count,lis_drug_cost,lis_suppress_flag,mapd_claim_count,mapd_drug_cost,mapd_suppress_flag,medicare_prvdr_enroll_status,nonlis_claim_count,nonlis_drug_cost,nonlis_suppress_flag,npi,nppes_credentials,nppes_entity_code,nppes_provider_city,nppes_provider_country,nppes_provider_first_name,nppes_provider_gender,nppes_provider_last_org_name,nppes_provider_mi,nppes_provider_state,nppes_provider_street1,nppes_provider_street2,nppes_provider_zip4,nppes_provider_zip5,opioid_bene_count,opioid_claim_count,opioid_day_supply,opioid_drug_cost,opioid_prescriber_rate,other_claim_count,other_drug_cost,other_suppress_flag,pdp_claim_count,pdp_drug_cost,pdp_suppress_flag,specialty_description,total_30_day_fill_count,total_30_day_fill_count_ge65,total_claim_count,total_claim_count_ge65,total_day_supply,total_day_supply_ge65,total_drug_cost,total_drug_cost_ge65
0,NaN,45.0,83.0,901.43,*,14.0,208.18,NaN,61.0,115.0,51.0,NaN,39.0,NaN,NaN,64.0,1.5820,115.0,58.0,57.0,0.0,0.0,100.0,NaN,0.0,0.0,NaN,374.0,67923.15,NaN,S,NaN,1679.0,24267.72,NaN,NaN,NaN,NaN,NaN,NaN,2088.0,92916.22,NaN,496.0,32965.73,NaN,E,0.0,0.00,NaN,1003005661,MD,I,WASHINGTON,US,ADEFOLAJU,F,OKETOKUN,NaN,DC,1818 NEW YORK AVE NE,SUITE 110,1848.0,20002,11.0,54.0,940.0,2837.15,2.59,35.0,725.35,NaN,1592.0,59950.49,NaN,General Practice,2151.7,1114.933333,2088,1082.0,58225,30784.0,92916.22,40171.69
1,0.0,31.0,45.0,633.64,NaN,0.0,0.00,NaN,68.0,98.0,65.0,NaN,43.0,NaN,NaN,33.0,3.9418,82.0,48.0,50.0,16.0,NaN,87.0,NaN,0.0,NaN,NaN,NaN,NaN,#,S,NaN,1540.0,24148.41,NaN,0.0,0.0,0.0,0.0,0.0,1736.0,182449.88,NaN,240.0,20270.14,NaN,E,181.0,4239.86,NaN,1003012063,MD,I,WASHINGTON,US,UZOAMAKA,F,NWAOGWUGWU,T,DC,2041 GEORGIA AVENUE NW,NaN,1.0,20060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*,1677.0,166419.60,NaN,Nephrology,2316.0,1639.000000,1917,1330.0,66810,47393.0,186689.74,75679.18
2,0.0,NaN,12.0,83.06,NaN,0.0,0.00,NaN,74.0,12.0,NaN,#,NaN,NaN,NaN,NaN,0.9154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.00,NaN,S,#,19.0,179.64,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,*,NaN,NaN,*,N,NaN,NaN,#,1003022831,D.D.S.,I,WASHINGTON,US,HASSAN,M,CHEHAYEB,E,DC,4201 MASSACHUSETTS AVE NW,1040C,4701.0,20016,NaN,NaN,NaN,NaN,NaN,0.0,0.00,NaN,NaN,NaN,#,Dentist,19.5,NaN,19,NaN,202,NaN,179.64,NaN
3,0.0,44.0,47.0,693.68,NaN,0.0,0.00,NaN,70.0,155.0,114.0,NaN,52.0,31.0,31.0,41.0,1.7831,66.0,97.0,58.0,89.0,NaN,24.0,16.0,NaN,NaN,104.0,NaN,NaN,#,S,NaN,196.0,2985.32,NaN,0.0,0.0,0.0,0.0,0.0,117.0,4047.73,NaN,208.0,7536.81,NaN,E,123.0,4215.32,NaN,1003042474,MD,I,WASHINGTON,US,EMILY,F,SIEGEL,J,DC,110 IRVING ST NW,NaN,3017.0,20010,30.0,30.0,123.0,341.47,12.50,NaN,NaN,*,32.0,726.24,NaN,Emergency Medicine,248.9,177.600000,240,171.0,3651,2484.0,8263.05,5303.27
4,0.0,11.0,16.0,253.79,NaN,0.0,0.00,NaN,71.0,71.0,60.0,NaN,35.0,NaN,NaN,11.0,1.6932,37.0,45.0,26.0,34.0,0.0,36.0,NaN,0.0,NaN,24.0,58.0,12113.23,NaN,S,NaN,345.0,5939.76,NaN,0.0,0.0,0.0,0.0,0.0,235.0,12292.14,NaN,22.0,672.60,NaN,E,168.0,5760.85,NaN,1003059437,M.D.,I,WASHINGTON,US,SRUTHI,F,REDDY,N,DC,3800 RESERVOIR ROAD NW,6 PHC,NaN,20007,NaN,NaN,

In [21]:

ps_df = ps_df.set_index("npi")

ps_df.head(10)

,anti_psych_bene_count_ge65,antibiotic_bene_count,antibiotic_claim_count,antibiotic_drug_cost,antipsych_bene_ge65_suppress_flg,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,antipsych_ge65_suppress_flag,average_age_of_beneficiaries,bene_count,bene_count_ge65,bene_count_ge65_suppress_flag,beneficiary_age_65_74_count,beneficiary_age_75_84_count,beneficiary_age_greater_84_count,beneficiary_age_less_65_count,beneficiary_average_risk_score,beneficiary_dual_count,beneficiary_female_count,beneficiary_male_count,beneficiary_nondual_count,beneficiary_race_asian_pi_count,beneficiary_race_black_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_race_white_count,brand_claim_count,brand_drug_cost,brand_suppress_flag,description_flag,ge65_suppress_flag,generic_claim_count,generic_drug_cost,generic_suppress_flag,la_opioid_bene_count,la_opioid_claim_count,la_opioid_day_supply,la_opioid_drug_cost,la_opioid_prescriber_rate,lis_claim_count,lis_drug_cost,lis_suppress_flag,mapd_claim_count,mapd_drug_cost,mapd_suppress_flag,medicare_prvdr_enroll_status,nonlis_claim_count,nonlis_drug_cost,nonlis_suppress_flag,nppes_credentials,nppes_entity_code,nppes_provider_city,nppes_provider_country,nppes_provider_first_name,nppes_provider_gender,nppes_provider_last_org_name,nppes_provider_mi,nppes_provider_state,nppes_provider_street1,nppes_provider_street2,nppes_provider_zip4,nppes_provider_zip5,opioid_bene_count,opioid_claim_count,opioid_day_supply,opioid_drug_cost,opioid_prescriber_rate,other_claim_count,other_drug_cost,other_suppress_flag,pdp_claim_count,pdp_drug_cost,pdp_suppress_flag,specialty_description,total_30_day_fill_count,total_30_day_fill_count_ge65,total_claim_count,total_claim_count_ge65,total_day_supply,total_day_supply_ge65,total_drug_cost,total_drug_cost_ge65
npi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1003005661,NaN,45.0,83.0,901.43,*,14.0,208.18,NaN,61.0,115.0,51.0,NaN,39.0,NaN,NaN,64.0,1.5820,115.0,58.0,57.0,0.0,0.0,100.0,NaN,0.0,0.0,NaN,374.0,67923.15,NaN,S,NaN,1679.0,24267.72,NaN,NaN,NaN,NaN,NaN,NaN,2088.0,92916.22,NaN,496.0,32965.73,NaN,E,0.0,0.00,NaN,MD,I,WASHINGTON,US,ADEFOLAJU,F,OKETOKUN,NaN,DC,1818 NEW YORK AVE NE,SUITE 110,1848.0,20002,11.0,54.0,940.0,2837.15,2.59,35.0,725.35,NaN,1592.0,59950.49,NaN,General Practice,2151.7,1114.933333,2088,1082.0,58225,30784.0,92916.22,40171.69
1003012063,0.0,31.0,45.0,633.64,NaN,0.0,0.00,NaN,68.0,98.0,65.0,NaN,43.0,NaN,NaN,33.0,3.9418,82.0,48.0,50.0,16.0,NaN,87.0,NaN,0.0,NaN,NaN,NaN,NaN,#,S,NaN,1540.0,24148.41,NaN,0.0,0.0,0.0,0.0,0.0,1736.0,182449.88,NaN,240.0,20270.14,NaN,E,181.0,4239.86,NaN,MD,I,WASHINGTON,US,UZOAMAKA,F,NWAOGWUGWU,T,DC,2041 GEORGIA AVENUE NW,NaN,1.0,20060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,*,1677.0,166419.60,NaN,Nephrology,2316.0,1639.000000,1917,1330.0,66810,47393.0,186689.74,75679.18
1003022831,0.0,NaN,12.0,83.06,NaN,0.0,0.00,NaN,74.0,12.0,NaN,#,NaN,NaN,NaN,NaN,0.9154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.00,NaN,S,#,19.0,179.64,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,*,NaN,NaN,*,N,NaN,NaN,#,D.D.S.,I,WASHINGTON,US,HASSAN,M,CHEHAYEB,E,DC,4201 MASSACHUSETTS AVE NW,1040C,4701.0,20016,NaN,NaN,NaN,NaN,NaN,0.0,0.00,NaN,NaN,NaN,#,Dentist,19.5,NaN,19,NaN,202,NaN,179.64,NaN
1003042474,0.0,44.0,47.0,693.68,NaN,0.0,0.00,NaN,70.0,155.0,114.0,NaN,52.0,31.0,31.0,41.0,1.7831,66.0,97.0,58.0,89.0,NaN,24.0,16.0,NaN,NaN,104.0,NaN,NaN,#,S,NaN,196.0,2985.32,NaN,0.0,0.0,0.0,0.0,0.0,117.0,4047.73,NaN,208.0,7536.81,NaN,E,123.0,4215.32,NaN,MD,I,WASHINGTON,US,EMILY,F,SIEGEL,J,DC,110 IRVING ST NW,NaN,3017.0,20010,30.0,30.0,123.0,341.47,12.50,NaN,NaN,*,32.0,726.24,NaN,Emergency Medicine,248.9,177.600000,240,171.0,3651,2484.0,8263.05,5303.27
1003059437,0.0,11.0,16.0,253.79,NaN,0.0,0.00,NaN,71.0,71.0,60.0,NaN,35.0,NaN,NaN,11.0,1.6932,37.0,45.0,26.0,34.0,0.0,36.0,NaN,0.0,NaN,24.0,58.0,12113.23,NaN,S,NaN,345.0,5939.76,NaN,0.0,0.0,0.0,0.0,0.0,235.0,12292.14,NaN,22.0,672.60,NaN,E,168.0,5760.85,NaN,M.D.,I,WASHINGT

In [22]:
ps_df = ps_df.fillna(0) #if the column doesn't normally take integer/float input, it was still replaced by 0.0


In [23]:
ps_df.head(10)

,anti_psych_bene_count_ge65,antibiotic_bene_count,antibiotic_claim_count,antibiotic_drug_cost,antipsych_bene_ge65_suppress_flg,antipsych_claim_count_ge65,antipsych_drug_cost_ge65,antipsych_ge65_suppress_flag,average_age_of_beneficiaries,bene_count,bene_count_ge65,bene_count_ge65_suppress_flag,beneficiary_age_65_74_count,beneficiary_age_75_84_count,beneficiary_age_greater_84_count,beneficiary_age_less_65_count,beneficiary_average_risk_score,beneficiary_dual_count,beneficiary_female_count,beneficiary_male_count,beneficiary_nondual_count,beneficiary_race_asian_pi_count,beneficiary_race_black_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_race_white_count,brand_claim_count,brand_drug_cost,brand_suppress_flag,description_flag,ge65_suppress_flag,generic_claim_count,generic_drug_cost,generic_suppress_flag,la_opioid_bene_count,la_opioid_claim_count,la_opioid_day_supply,la_opioid_drug_cost,la_opioid_prescriber_rate,lis_claim_count,lis_drug_cost,lis_suppress_flag,mapd_claim_count,mapd_drug_cost,mapd_suppress_flag,medicare_prvdr_enroll_status,nonlis_claim_count,nonlis_drug_cost,nonlis_suppress_flag,nppes_credentials,nppes_entity_code,nppes_provider_city,nppes_provider_country,nppes_provider_first_name,nppes_provider_gender,nppes_provider_last_org_name,nppes_provider_mi,nppes_provider_state,nppes_provider_street1,nppes_provider_street2,nppes_provider_zip4,nppes_provider_zip5,opioid_bene_count,opioid_claim_count,opioid_day_supply,opioid_drug_cost,opioid_prescriber_rate,other_claim_count,other_drug_cost,other_suppress_flag,pdp_claim_count,pdp_drug_cost,pdp_suppress_flag,specialty_description,total_30_day_fill_count,total_30_day_fill_count_ge65,total_claim_count,total_claim_count_ge65,total_day_supply,total_day_supply_ge65,total_drug_cost,total_drug_cost_ge65
npi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1003005661,0.0,45.0,83.0,901.43,*,14.0,208.18,0,61.0,115.0,51.0,0,39.0,0.0,0.0,64.0,1.5820,115.0,58.0,57.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,374.0,67923.15,0,S,0,1679.0,24267.72,0,0.0,0.0,0.0,0.0,0.0,2088.0,92916.22,0,496.0,32965.73,0,E,0.0,0.00,0,MD,I,WASHINGTON,US,ADEFOLAJU,F,OKETOKUN,0,DC,1818 NEW YORK AVE NE,SUITE 110,1848.0,20002,11.0,54.0,940.0,2837.15,2.59,35.0,725.35,0,1592.0,59950.49,0,General Practice,2151.7,1114.933333,2088,1082.0,58225,30784.0,92916.22,40171.69
1003012063,0.0,31.0,45.0,633.64,0,0.0,0.00,0,68.0,98.0,65.0,0,43.0,0.0,0.0,33.0,3.9418,82.0,48.0,50.0,16.0,0.0,87.0,0.0,0.0,0.0,0.0,0.0,0.00,#,S,0,1540.0,24148.41,0,0.0,0.0,0.0,0.0,0.0,1736.0,182449.88,0,240.0,20270.14,0,E,181.0,4239.86,0,MD,I,WASHINGTON,US,UZOAMAKA,F,NWAOGWUGWU,T,DC,2041 GEORGIA AVENUE NW,0,1.0,20060,0.0,0.0,0.0,0.00,0.00,0.0,0.00,*,1677.0,166419.60,0,Nephrology,2316.0,1639.000000,1917,1330.0,66810,47393.0,186689.74,75679.18
1003022831,0.0,0.0,12.0,83.06,0,0.0,0.00,0,74.0,12.0,0.0,#,0.0,0.0,0.0,0.0,0.9154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0,S,#,19.0,179.64,0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,*,0.0,0.00,*,N,0.0,0.00,#,D.D.S.,I,WASHINGTON,US,HASSAN,M,CHEHAYEB,E,DC,4201 MASSACHUSETTS AVE NW,1040C,4701.0,20016,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0,0.0,0.00,#,Dentist,19.5,0.000000,19,0.0,202,0.0,179.64,0.00
1003042474,0.0,44.0,47.0,693.68,0,0.0,0.00,0,70.0,155.0,114.0,0,52.0,31.0,31.0,41.0,1.7831,66.0,97.0,58.0,89.0,0.0,24.0,16.0,0.0,0.0,104.0,0.0,0.00,#,S,0,196.0,2985.32,0,0.0,0.0,0.0,0.0,0.0,117.0,4047.73,0,208.0,7536.81,0,E,123.0,4215.32,0,MD,I,WASHINGTON,US,EMILY,F,SIEGEL,J,DC,110 IRVING ST NW,0,3017.0,20010,30.0,30.0,123.0,341.47,12.50,0.0,0.00,*,32.0,726.24,0,Emergency Medicine,248.9,177.600000,240,171.0,3651,2484.0,8263.05,5303.27
1003059437,0.0,11.0,16.0,253.79,0,0.0,0.00,0,71.0,71.0,60.0,0,35.0,0.0,0.0,11.0,1.6932,37.0,45.0,26.0,34.0,0.0,36.0,0.0,0.0,0.0,24.0,58.0,12113.23,0,S,0,345.0,5939.76,0,0.0,0.0,0.0,0.0,0.0,235.0,12292.14,0,22.0,672.60,0,E,168.0,5760.85,0,M.D.,I,WASHINGTON,US,SRUTHI,F,REDDY,N,DC,3800 RESERVOIR ROAD NW,6 PHC,0.0,20007,0.0,0.0

In [24]:
ps_df_new = ps_df[["specialty_description", "brand_claim_count", "brand_drug_cost", "generic_claim_count", "generic_drug_cost", "nppes_provider_city", "total_claim_count", "total_drug_cost"]]

In [34]:

ps_df_new.head(10)

,specialty_description,brand_claim_count,brand_drug_cost,generic_claim_count,generic_drug_cost,nppes_provider_city,total_claim_count,total_drug_cost
npi,,,,,,,,
1003005661,General Practice,374.0,67923.15,1679.0,24267.72,WASHINGTON,2088,92916.22
1003012063,Nephrology,0.0,0.00,1540.0,24148.41,WASHINGTON,1917,186689.74
1003022831,Dentist,0.0,0.00,19.0,179.64,WASHINGTON,19,179.64
1003042474,Emergency Medicine,0.0,0.00,196.0,2985.32,WASHINGTON,240,8263.05
1003059437,Internal Medicine,58.0,12113.23,345.0,5939.76,WASHINGTON,403,18052.99
1003067778,Dentist,0.0,0.00,17.0,77.65,WASHINGTON,17,77.65
1003098351,Orthopedic Surgery,0.0,0.00,0.0,0.00,WASHINGTON,139,5458.21
1003102831,Student in an Organized Health Care Education/...,0.0,0.00,350.0,4954.71,WASHINGTON,456,31508.10
1003103151,Dentist,0.0,0.00,27.0,111.72,WASHINGTON,27,111.72


In [35]:
ps_df_new.shape

(4880, 8)

In [29]:
ps_dd_df= pd.concat([ps_df_new, npi_drug_cross_df], axis = 1, join="inner")

In [30]:
ps_dd_df.head(10)

,specialty_description,brand_claim_count,brand_drug_cost,generic_claim_count,generic_drug_cost,nppes_provider_city,total_claim_count,total_drug_cost,ABACAVIR,ABACAVIR-LAMIVUDINE,ABACAVIR-LAMIVUDINE-ZIDOVUDINE,ABILIFY MAINTENA,ACAMPROSATE CALCIUM,ACARBOSE,ACCUPRIL,ACEBUTOLOL HCL,ACETAMINOPHEN-CODEINE,ACETAZOLAMIDE,ACTEMRA,ACYCLOVIR,ADACEL TDAP,ADAPALENE,ADCIRCA,ADDERALL,ADDERALL XR,ADEFOVIR DIPIVOXIL,ADEMPAS,ADVAIR DISKUS,ADVAIR HFA,AGGRENOX,ALBUTEROL SULFATE,ALCLOMETASONE DIPROPIONATE,ALCOHOL PADS,ALCOHOL PREP PADS,ALENDRONATE SODIUM,ALFUZOSIN HCL ER,ALLOPURINOL,ALOSETRON HCL,ALPHAGAN P,ALPRAZOLAM,ALPRAZOLAM ER,ALTAVERA,AMANTADINE,AMBIEN CR,AMIKACIN SULFATE,AMILORIDE HCL,AMIODARONE HCL,AMITIZA,AMITRIPTYLINE HCL,AMLODIPINE BESYLATE,AMLODIPINE BESYLATE-BENAZEPRIL,AMLODIPINE-ATORVASTATIN,AMLODIPINE-OLMESARTAN,AMLODIPINE-VALSARTAN,AMLODIPINE-VALSARTAN-HCTZ,AMMONIUM LACTATE,AMOXAPINE,AMOXICILLIN,AMOXICILLIN-CLAVULANATE POTASS,AMPICILLIN TRIHYDRATE,AMPYRA,ANAGRELIDE HCL,ANASTROZOLE,ANDROGEL,ANORO ELLIPTA,APRISO,APTIOM,ARANESP,ARIPIPRAZOLE,ARISTADA,ARMODAFINIL,ARMOUR THYROID,ARNUITY ELLIPTA,ASCOMP WITH CODEINE,ASMANEX,...,UNITHROID,UPTRAVI,URSODIOL,VAGIFEM,VALACYCLOVIR,VALGANCICLOVIR HCL,VALIUM,VALPROIC ACID,VALSARTAN,VALSARTAN-HYDROCHLOROTHIAZIDE,VANCOMYCIN,VANCOMYCIN HCL,VASCEPA,VELPHORO,VEMLIDY,VENCLEXTA,VENLAFAXINE HCL,VENLAFAXINE HCL ER,VENTOLIN HFA,VERAPAMIL ER,VERAPAMIL ER PM,VERAPAMIL HCL,VERAPAMIL SR,VESICARE,VIBERZI,VICTOZA 2-PAK,VICTOZA 3-PAK,VIGAMOX,VIIBRYD,VIMPAT,VIRACEPT,VIREAD,VIVELLE-DOT,VIVITROL,VOLTAREN,VOTRIENT,VRAYLAR,VYVANSE,WARFARIN SODIUM,WATER,WELCHOL,WELLBUTRIN XL,XALATAN,XALKORI,XANAX,XARELTO,XELJANZ,XELJANZ XR,XENAZINE,XIFAXAN,XIGDUO XR,XOLAIR,XOPENEX HFA,XTANDI,YUVAFEM,ZALEPLON,ZARXIO,ZENPEP,ZEPATIER,ZERBAXA,ZETIA,ZIDOVUDINE,ZINBRYTA,ZIOPTAN,ZIPRASIDONE HCL,ZOLOFT,ZOLPIDEM TARTRATE,ZOLPIDEM TARTRATE ER,ZONISAMIDE,ZOSTAVAX,ZUBSOLV,ZYDELIG,ZYPREXA,ZYPREXA RELPREVV,ZYTIGA
npi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1003005661,General Practice,374.0,67923.15,1679.0,24267.72,WASHINGTON,2088,92916.22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,138.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003012063,Nephrology,0.0,0.00,1540.0,24148.41,WASHINGTON,1917,186689.74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,176.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003042474,Emergency Medicine,0.0,0.00,196.0,2985.32,WASHINGTON,240,8263.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [32]:
ps_dd_df.shape

(2984, 1117)

ratios --> visualization techniques -> courses / online
read up on regression
What data is missing, distributions, transformations?
write what direction/going to do
GO JOHN